In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

import pickle
import os
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [6]:
from google.colab import files
uploaded = files.upload()

Saving data.zip to data.zip


In [12]:
!! unzip data.zip

['Archive:  data.zip',
 '   creating: data/',
 '  inflating: data/nice_embed_tsne.csv  ',
 '  inflating: data/acid_properties.csv  ',
 '  inflating: data/family_classification_sequences.tab  ',
 '  inflating: data/family_classification_metadata.tab  ']

In [14]:
!ls data

acid_properties.csv		    family_classification_sequences.tab
family_classification_metadata.tab  nice_embed_tsne.csv


In [15]:
seq_df = pd.read_table('data/family_classification_sequences.tab')
seq_df.head()

,Sequences
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...


In [0]:
index = 0
codone_dict = {}

def index_dict(codone):
      global index
      if codone in codone_dict:
          return codone_dict[codone]

      codone_dict[codone] = index
      index += 1
      return index - 1

def make_codones(sseq):
    crop = len(sseq) % 3
    cropped_seq = sseq[:-crop] if crop > 0 else sseq

    return [index_dict(cropped_seq[i:i+3]) for i in range(0, len(cropped_seq), 3)]

def seq_to3(seq):
    splittings = [make_codones(seq[i:]) for i in range(3)]
    return splittings

def create_all_codones(df):
    codones = []

    for i in range(df.shape[0]):
        row = df.iloc[i, :][0]
        codones.extend(seq_to3(row))
    return codones
  
def read_or_create(read_path, producer):
    if os.path.isfile(read_path):
        print('reading', read_path)
        with open(read_path, 'rb') as fp:
            return pickle.load(fp)
    result = producer()
    print('saving', read_path)
    with open(read_path, 'wb') as fp:
        pickle.dump(result, fp)
    return result

In [19]:
all_codones = read_or_create(read_path='data/all_codones.pickle',
                             producer= lambda: create_all_codones(seq_df))

saving data/all_codones.pickle


In [0]:
######################

In [0]:
def generate_sample(index_words_list, context_window_size):
    """ Form training pairs according to the skip-gram model. """
    for index_words in index_words_list:
        for index, center in enumerate(index_words):
            context = random.randint(1, context_window_size)
            # get a random target before the center word
            for target in index_words[max(0, index - context): index]:
                yield center, target
            # get a random target after the center wrod
            for target in index_words[index + 1: index + context + 1]:
                yield center, target


def get_batch(iterator, batch_size):
    """ Group a numerical stream into batches and yield them as Numpy arrays. """
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros(batch_size, dtype=np.int32)
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(iterator)
        yield center_batch, target_batch


def flatten(x):
    return [item for sublist in x for item in sublist]


def cod_to_dict(cod, dictionary):
    return [dictionary[key] for key in cod]

def make_dictionary(all_codones):
    flat_codones = flatten(all_codones)
    unique_codones = set(flat_codones)
    dictionary = {cod: i for i, cod in enumerate(unique_codones)}
    return dictionary

def process_data(all_codones, dictionary, batch_size, skip_window):
    cod_dicts = [cod_to_dict(cod, dictionary) for cod in all_codones]
    single_gen = generate_sample(cod_dicts, context_window_size=skip_window)
    batch_gen = get_batch(single_gen, batch_size=batch_size)
    return batch_gen

In [0]:
dictionary = make_dictionary(all_codones)

def create_freq(all_codones, dictionary):
    flat_codones = flatten(all_codones)
    freq = {}
    
    for codone in flat_codones:
        if dictionary[codone] not in freq:
            freq[dictionary[codone]] = 1
        else:
            freq[dictionary[codone]] += 1
        
    return freq


def create_unigram(freq):
    freq_values = list(freq.values())
    freq_values = torch.FloatTensor(freq_values)
    
    distr = freq_values / freq_values.sum()
    distr = distr.pow(3.0 / 4.0)
    distr = distr / distr.sum()
    return distr.numpy()
    

freq_dict = create_freq(all_codones, dictionary)
distr = create_unigram(freq_dict)

In [0]:
BATCH_SIZE = 512
SKIP_WINDOW = 12  # the context window

batch_gen = process_data(all_codones, dictionary, BATCH_SIZE, SKIP_WINDOW)

In [0]:
######################

In [0]:
class SkipGramModel(nn.Module):
    
  def __init__(self, vocab_size, embed_size):
      super(SkipGramModel, self).__init__()
      self.vocab_size = vocab_size
      self.embed_size = embed_size

      self.center_embs = nn.Embedding(vocab_size, embed_size)
      self.context_embs = nn.Embedding(vocab_size, embed_size)

  def forward(self, centers, contexts, negative):
      batch_size = centers.size()[0]

      cntr = self.center_embs(centers)
      pos_ctx = self.context_embs(contexts)
      neg_ctx = self.context_embs(negative)

      pos_mul = torch.bmm(cntr.unsqueeze(1), pos_ctx.unsqueeze(2)).squeeze()
      pos_loss = F.logsigmoid(pos_mul).sum()

      neg_mul = torch.bmm(neg_ctx, cntr.unsqueeze(2)).squeeze().sum(dim=1)
      neg_loss = F.logsigmoid(-neg_mul).sum()

      return -(pos_loss + neg_loss) / batch_size


In [0]:
VOCAB_SIZE = 9424
EMBED_SIZE = 100  # dimension of the word embedding vectors
NUM_SAMPLED = 5  # Number of negative examples to sample.
LEARNING_RATE = .9
NUM_TRAIN_STEPS = 100000
SKIP_STEP = 2000

LOGGER = SummaryWriter()

def train_embeddings(batch_gen):        
    torch.manual_seed(1)
    np.random.seed(1)

    model = SkipGramModel(VOCAB_SIZE, EMBED_SIZE)
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.train()
    model.to(device)
    
    for iter in range(NUM_TRAIN_STEPS):
        avg_loss = 0.
      
        centers, context = next(batch_gen)
        negative = np.random.choice(VOCAB_SIZE, size=(len(centers), NUM_SAMPLED), p=distr)
    
        centers = torch.tensor(centers, dtype=torch.long).to(device)
        context = torch.tensor(context, dtype=torch.long).to(device)
        negative = torch.tensor(negative, dtype=torch.long).to(device)

        optimizer.zero_grad()
        loss = model.forward(centers, context, negative)
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()

        LOGGER.add_scalar('loss', loss, iter + 1)
        if iter % SKIP_STEP == 0:
            print("Iteration # {}, loss == {:5.3f}".format(iter + 1, avg_loss))
        
    
    embeddings =  model.center_embs(torch.tensor([w for w in range(self.vocab_size)], dtype=torch.long).to(device)).detach().cpu().numpy()
    LOGGER.add_embedding(embeddings)
    
    return embeddings

In [0]:
final_embed_matrix = read_or_create(read_path='data/embeddings.pickle',
                                    producer=lambda: train_embeddings(batch_gen))

Iteration # 1, loss == 13.900
Iteration # 2001, loss == 10.865
Iteration # 4001, loss == 11.413
Iteration # 6001, loss == 8.672
Iteration # 8001, loss == 8.849
Iteration # 10001, loss == 5.108
Iteration # 12001, loss == 6.146


In [0]:
######################

In [0]:
tsne = TSNE(n_components=2, random_state=42)
XX = tsne.fit_transform(final_embed_matrix)

In [0]:
tsne_df = pd.DataFrame(XX, columns=['x0', 'x1'])
unique_codones = sorted(dictionary, key=dictionary.get)
tsne_df['codone'] = list(unique_codones)
tsne_df.head()

In [0]:
def plot_tsne_df(df):
    plt.figure(figsize=(15, 10))
    plt.title('unlabeled encoding', fontsize=20)
    plt.scatter(df.x0, df.x1, s=10)
    plt.show()
    
plot_tsne_df(tsne_df)

In [0]:
filename = 'data/acid_properties.csv'
props = pd.read_csv(filename) 

In [0]:
######################

In [0]:
def acid_dict(some_c, props):
    prop_by_letter = [props[props.acid == let].iloc[:, 1:] for let in some_c]   
    df_concat = pd.concat(prop_by_letter)
    res = df_concat.mean()
    dres = dict(res)
    dres['acid'] = some_c
    return dres

In [0]:
save_path = 'data/all_acid_dicts.pickle'
producer = lambda: [acid_dict(some_c, props) for some_c in tsne_df.codone]
all_acid_dicts = read_or_create(save_path, producer)

In [0]:
all_acid_df = pd.DataFrame(all_acid_dicts)
all_acid_df.head()

In [0]:
final_df = all_acid_df.join(tsne_df.set_index('codone'), on='acid')
final_df.head()

In [0]:
def plot_embedding_properties(final_df):
    plt.figure(figsize=(25, 20))
    for i, p in enumerate(['hydrophobicity', 'mass', 'number_of_atoms', 'volume']):
        plt.subplot(2,2,i+1)
        plt.title(p, fontsize=25)
        plt.scatter(final_df.x0, final_df.x1, c=final_df[p], s=10)
    plt.show()

In [0]:
plot_embedding_properties(final_df)

In [0]:
######################

In [0]:
filename = 'data/nice_embed_tsne.csv'
gensim_tsne_df = pd.read_csv(filename, index_col=0)
gensim_tsne_df.columns = ['x0', 'x1', 'codone']

In [0]:
plot_tsne_df(gensim_tsne_df)

In [0]:
final_df_nice = all_acid_df.join(gensim_tsne_df.set_index('codone'), on='acid')

In [0]:
plot_embedding_properties(final_df_nice)

## Homework

* Implement in Pytorch and fine-tune this SkipGramModel to archive better embedding for amino acids codones. 
* Visualize your space in the similar style as on the bottom example. 
* Visualize 3D T-SNE in TensorboardX

Article with the original research can be found here http://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0141287&type=printable

Bonus task(no credit): visualize your embedding space in similar manner as minst example: https://www.tensorflow.org/versions/r0.12/how_tos/embedding_viz/

In [0]:
soft deadline: 14.10.2018 at 23.59

hard deadline: 17.01.2018 at 23.59